<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/CNN_HandWriting_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 데이터 전처리 : 채널 하나 더 만들고, 0~1 로 정규화
train_images = train_images / 255.0
train_images = train_images.reshape(train_images.shape[0], 28, 28 ,1).astype('float32')

test_images = test_images / 255.0
test_images = test_images.reshape(test_images.shape[0], 28, 28 ,1).astype('float32')

# 데이터 배치 만들고 섞기
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [ ]:
# 모델 정의

class MyCNNModel(tf.keras.Model):
  def __init__(self):
    super(MyCNNModel, self).__init__()
    self.conv1 = layers.Conv2D(32, 3, padding = 'same')
    self.conv2 = layers.Conv2D(64, 3, padding = 'same')
    self.conv3 = layers.Conv2D(128, 3, padding = 'same')
    self.flatten = layers.Flatten()
    self.batchnorm = layers.BatchNormalization()
    self.lrelu = layers.LeakyReLU()
    self.dropout = layers.Dropout(0.3)
    self.d1 = layers.Dense(128, activation = 'relu')
    self.d2 = layers.Dense(10)

  def call(self, x):
    x = self.conv1(x) # 함수형으로 만들면 input_shape 설정 필요 없나??
    # x = self.batchnorm(x)
    x = self.lrelu(x)
    x = self.dropout(x)
    x = self.conv2(x)
    # x = self.batchnorm(x)
    x = self.lrelu(x)
    x = self.dropout(x)
    x = self.conv3(x)
    # x = self.batchnorm(x)
    x = self.lrelu(x)
    x = self.dropout(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyCNNModel()
# def myCNNModel():
#   model = tf.keras.Sequential()
#   model.add(layers.Conv2D(32, (5, 5), strides = (2,2), padding = 'same', input_shape = [28, 28, 1]))
#   model.add(layers.LeakyReLU())
#   model.add(layers.Dropout(0.3))

#   model.add(layers.Conv2D(64, (5, 5), strides = (2,2), padding = 'same'))
#   model.add(layers.LeakyReLU())
#   model.add(layers.Dropout(0.3))

#   model.add(layers.Conv2D(128, (5, 5), strides = (2,2), padding = 'same'))
#   model.add(layers.LeakyReLU())
#   model.add(layers.Dropout(0.3))

#   model.add(layers.Flatten())
#   model.add(layers.Dense(10))

#   return model

In [ ]:
# 옵티마이저 & 손실 함수 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam()

# # 체크포인트 & 매니저 정의
# ckpt = tf.train.Checkpoint(step = tf.Variable(1), optimizer = optimizer, net = model)
# manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep = 5)

In [ ]:
# 모델 손실, 성능 측정 지표
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_acc')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_acc')


In [ ]:
# 모델 훈련
@tf.function 
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training = True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(labels, predictions)
  return loss

In [ ]:
# 훈련 정보 저장
# def train_and_checkpoint(manager):
#   ckpt.restore(manager.latest_checkpoint)
#   if manager.latest_checkpoint == False:
#     print("가져올 모델이 없음")

#   for images, labels in train_dataset:
#     loss = train_step(images, labels)
#     ckpt.step.assign_add(1)
#     if int(ckpt.step) % 5 == 0:
#       save_path = manager.save()
#       print(f"스텝 {int(ckpt.step)}의 값이 {save_path}에 저장됨.")
#       print('loss :{:1.2f}'.format(loss.numpy()))

In [ ]:
# train_and_checkpoint(manager)

In [ ]:
# 추론
@tf.function
def test_step(images, labels):
  predictions = model(images, 
                      training = False) # Dropout 같은 레이어는 자동으로 꺼짐
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_acc(labels, predictions)

In [ ]:
# 전체 훈련 과정

def train(EPOCHS = 10):

  for epoch in range(1, EPOCHS + 1):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

    for images, labels in train_dataset:
      train_step(images, labels)

    # 에포크에 대한 체크포인트를 작성하려면 여기에서 해야 함 - 체크포인트 저장
    if epoch % 5 == 0 :
      model.save_weights(f'save_epoch_{epoch}')
      print(f'에포크 {epoch}의 모델 저장됨.')

    for test_images, test_labels in test_dataset:
      test_step(test_images, test_labels)

    print(f'Epoch {epoch}', 
          f'Loss : {train_loss.result()}',
          "Accuracy : {:.2f}%".format(train_acc.result() * 100),
          f'Test Loss : {test_loss.result()}',
          'Test Accuracy : {:.2f}'.format(test_acc.result() * 100)
          )

train(20)

In [ ]:
# 체크포인트는 가중치만을 저장함 
# 모델 자체로 저장하면 모델을 다시 정의할 필요 없이 이용할 수 있음 (h5)

# 근데 SubClass Model을 이용한 경우는 h5로 저장이 불가능함 - 파이썬의 방법이기 떄문에 안전하게 Serialize할 수 없다는 듯
# 텐서플로우에서는 TensorFlow SavedModel 포맷이나 save_weights를 쓸 것을 권고함
model.save('my_model.tf') 
model.summary()
# 모델 로드:
# new_model = tf.keras.models.load_model('my_model.tf')
# new_model.summary()

In [ ]:
# 암만 봐도 그냥 케라스 쓰는게 훨씬 편한..

# connect google drive
from google.colab import drive
drive.mount('/content/drive')